In [21]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:

path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK080732,Karma Resorts,Other,Past Guests,Karma Kandara,PF331628,Ms,Angela,Everhart,Female,...,2025-11-04 00:00:00,7,COMP,GSTAFF,COMP,1,0,0,Email Only,NO
1,KK080346,Karma Resorts,Other,Past Guests,Karma Kandara,PF330578,Mr.,John Raymont,Taylor,Male,...,2025-11-03 00:00:00,2,HOUSE_USE,GSTAFF,COMP,2,0,0,Email and Phone,NO
2,KK070689,Karma Resorts,Other,Past Guests,Karma Kandara,PF305542,Mr.,Paul Leslie,Clark,Male,...,2025-11-03 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
3,KB053982,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332753,Mrs,Jessica,Hu,Female,...,2025-11-07 00:00:00,5,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
4,KK064587,Karma Resorts,Other,Past Guests,Karma Kandara,PF293968,Mr,Michael,Robinson,Male,...,2025-11-03 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
5,KB054126,Karma Resorts,Other,Past Guests,Karma Bavaria,PF333458,Frau,Bianca,Metzner,Female,...,2025-11-07 00:00:00,4,DIGITAL,TEL,RETAIL,2,0,2,Email and Phone,NO
6,KB054467,Karma Resorts,Other,Past Guests,Karma Bavaria,PF335577,Frau,Caterina,Veiel,Unknown,...,2025-11-07 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
7,KB054467,Karma Resorts,Other,Past Guests,Karma Bavaria,PF335937,Frau,Nicola,Schönhofer,Female,...,2025-11-07 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
8,KB052517,Karma Resorts,Other,Past Guests,Karma Bavaria,PF251012,<NA>,Claudina,Gaber,Female,...,2025-11-02 00:00:00,1,OTA,OTA,RETAIL,2,2,0,Email and Phone,NO
9,KB051927,Karma Resorts,Other,Past Guests,Karma Bavaria,PF251012,<NA>,Claudina,Gaber,Female,...,2025-11-08 00:00:00,1,OTA,OTA,RETAIL,2,2,0,Email and Phone,NO


In [23]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [24]:
df_analytic.fillna('empty', inplace=True)

In [25]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [26]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [27]:
df_analytic.loc[
    (df_analytic['TelNo'] == 'empty') & (df_analytic['MobileNo'] != 'empty'), 
    'TelNo'] = df_analytic['MobileNo']
df_analytic.loc[df_analytic['TelNo'] == df_analytic['MobileNo'],'MobileNo'] = 'empty'

In [28]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
9,KB051927,Karma Resorts,Other,Past Guests,Karma Bavaria,PF251012,empty,Claudina,Gaber,Female,...,2025-11-08 00:00:00,1,OTA,OTA,RETAIL,2,2,0,Email and Phone,NO
17,KB052855,Karma Resorts,Other,Past Guests,Karma Bavaria,PF326402,Herr,Harald Und Helga,Brixner,Unknown,...,2025-11-08 00:00:00,6,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
18,KB054236,Karma Resorts,Other,Past Guests,Karma Bavaria,PF326402,Herr,Harald Und Helga,Brixner,Unknown,...,2025-11-05 00:00:00,3,COMP,GER_OWE,COMP,2,2,0,Email Only,NO
23,KB054482,Karma Resorts,Other,Past Guests,Karma Bavaria,PF335677,Herr,Marius,Strimbu,Male,...,2025-11-04 00:00:00,1,OTA,OTA,RETAIL,2,2,0,Email and Phone,NO
25,KB050144,Karma Resorts,Other,Past Guests,Karma Bavaria,PF065347,Frau,Erika,Schaffer,Female,...,2025-11-06 00:00:00,4,COMP,GER_OWE,COMP,4,0,0,Email and Phone,NO
33,KB051549,Karma Resorts,Other,Past Guests,Karma Bavaria,PF118055,Herr,Tabia,Karim,Male,...,2025-11-03 00:00:00,3,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
34,KB051548,Karma Resorts,Other,Past Guests,Karma Bavaria,PF118055,Herr,Tabia,Karim,Male,...,2025-11-03 00:00:00,3,COMP,GER_OWE,COMP,4,0,0,Email and Phone,NO
40,KB054063,Karma Resorts,Other,Past Guests,Karma Bavaria,PF290977,Herr,Werner,Wiemken,Male,...,2025-11-07 15:00:00,0,CORP,LGR,GROUP,1,0,0,Email and Phone,NO
47,KB052100,Karma Resorts,Other,Past Guests,Karma Bavaria,PF321976,empty,Tina,Grimm,Unknown,...,2025-11-03 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO


In [29]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK080732,Karma Resorts,Other,Past Guests,Karma Kandara,PF331628,Ms,Angela,Everhart,Female,...,2025-11-04 00:00:00,7,COMP,GSTAFF,COMP,1,0,0,Email Only,NO
1,KK080346,Karma Resorts,Other,Past Guests,Karma Kandara,PF330578,Mr.,John Raymont,Taylor,Male,...,2025-11-03 00:00:00,2,HOUSE_USE,GSTAFF,COMP,2,0,0,Email and Phone,NO
2,KK070689,Karma Resorts,Other,Past Guests,Karma Kandara,PF305542,Mr.,Paul Leslie,Clark,Male,...,2025-11-03 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
3,KB053982,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332753,Mrs,Jessica,Hu,Female,...,2025-11-07 00:00:00,5,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
4,KK064587,Karma Resorts,Other,Past Guests,Karma Kandara,PF293968,Mr,Michael,Robinson,Male,...,2025-11-03 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
5,KB054126,Karma Resorts,Other,Past Guests,Karma Bavaria,PF333458,Frau,Bianca,Metzner,Female,...,2025-11-07 00:00:00,4,DIGITAL,TEL,RETAIL,2,0,2,Email and Phone,NO
6,KB054467,Karma Resorts,Other,Past Guests,Karma Bavaria,PF335577,Frau,Caterina,Veiel,Unknown,...,2025-11-07 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
7,KB054467,Karma Resorts,Other,Past Guests,Karma Bavaria,PF335937,Frau,Nicola,Schönhofer,Female,...,2025-11-07 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
8,KB052517,Karma Resorts,Other,Past Guests,Karma Bavaria,PF251012,empty,Claudina,Gaber,Female,...,2025-11-02 00:00:00,1,OTA,OTA,RETAIL,2,2,0,Email and Phone,NO
9,KB053770,Karma Resorts,Other,Past Guests,Karma Bavaria,PF147772,Frau,Claudia,Goebl,Female,...,2025-11-05 00:00:00,3,COMP,GER_OWE,COMP,2,0,0,Email Only,NO


In [30]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [31]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [32]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
9,KB053770,Karma Resorts,Other,Past Guests,Karma Bavaria,PF147772,Frau,Claudia,Goebl,Female,...,2025-11-05 00:00:00,3,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
15,KB052854,Karma Resorts,Other,Past Guests,Karma Bavaria,PF326402,Herr,Harald Und Helga,Brixner,Unknown,...,2025-11-08 00:00:00,6,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
20,KB050145,Karma Resorts,Other,Past Guests,Karma Bavaria,PF065347,Frau,Erika,Schaffer,Female,...,2025-11-06 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
27,KB051638,Karma Resorts,Other,Past Guests,Karma Bavaria,PF118055,Herr,Tabia,Karim,Male,...,2025-11-03 00:00:00,3,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO


In [33]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [34]:
count_data_akhir = df_analytic.shape[0]

In [35]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB054382,Karma Resorts,Other,Past Guests,Karma Bavaria,PF335042,Herr,Patrick,Kuhbeck,Male,...,2025-11-02 00:00:00,1,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
1,KB053508,Karma Resorts,Other,Past Guests,Karma Bavaria,PF330126,Herr,Harald,Laue,Unknown,...,2025-11-02 00:00:00,2,OTA,OTA,RETAIL,4,2,0,Email and Phone,NO
2,KK082179,Karma Resorts,Other,Past Guests,Karma Kandara,PF078478,Mrs,Jofi,Sunjoto,Female,...,2025-11-02 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
3,KB054405,Karma Resorts,Other,Past Guests,Karma Bavaria,PF335168,Frau,Paula,Ciko,Female,...,2025-11-02 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
4,KB052517,Karma Resorts,Other,Past Guests,Karma Bavaria,PF251012,empty,Claudina,Gaber,Female,...,2025-11-02 00:00:00,1,OTA,OTA,RETAIL,2,2,0,Email and Phone,NO
5,KB054004,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332900,Frau,Tamara,Behr,Unknown,...,2025-11-02 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
6,KB054169,Karma Resorts,Other,Past Guests,Karma Bavaria,PF333735,Frau,Sofia,Smuda,Unknown,...,2025-11-02 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
7,KB054177,Karma Resorts,Other,Past Guests,Karma Bavaria,PF333840,Herr,Florian,Prokopec,Unknown,...,2025-11-02 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
8,KB053983,Karma Resorts,Other,Past Guests,Karma Bavaria,PF335548,Mr,Thomas,Hales,Male,...,2025-11-02 00:00:00,2,WYNDHAM,WYNDHAM,WYNDHAM,4,0,0,Email and Phone,NO
9,KB054317,Karma Resorts,Other,Past Guests,Karma Bavaria,PF334661,Frau,Melanie,Haubner,Unknown,...,2025-11-02 00:00:00,1,OTA,OTA,RETAIL,2,1,0,Email and Phone,NO


In [36]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [37]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_24876\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [38]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB054382,Karma Resorts,Other,Past Guests,Karma Bavaria,PF335042,Herr,Patrick,Kuhbeck,Male,...,2025-11-02,1,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
1,KB053508,Karma Resorts,Other,Past Guests,Karma Bavaria,PF330126,Herr,Harald,Laue,Unknown,...,2025-11-02,2,OTA,OTA,RETAIL,4,2,0,Email and Phone,NO
2,KK082179,Karma Resorts,Other,Past Guests,Karma Kandara,PF078478,Mrs,Jofi,Sunjoto,Female,...,2025-11-02,1,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
3,KB054405,Karma Resorts,Other,Past Guests,Karma Bavaria,PF335168,Frau,Paula,Ciko,Female,...,2025-11-02,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
4,KB052517,Karma Resorts,Other,Past Guests,Karma Bavaria,PF251012,,Claudina,Gaber,Female,...,2025-11-02,1,OTA,OTA,RETAIL,2,2,0,Email and Phone,NO
5,KB054004,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332900,Frau,Tamara,Behr,Unknown,...,2025-11-02,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
6,KB054169,Karma Resorts,Other,Past Guests,Karma Bavaria,PF333735,Frau,Sofia,Smuda,Unknown,...,2025-11-02,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
7,KB054177,Karma Resorts,Other,Past Guests,Karma Bavaria,PF333840,Herr,Florian,Prokopec,Unknown,...,2025-11-02,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
8,KB053983,Karma Resorts,Other,Past Guests,Karma Bavaria,PF335548,Mr,Thomas,Hales,Male,...,2025-11-02,2,WYNDHAM,WYNDHAM,WYNDHAM,4,0,0,Email and Phone,NO
9,KB054317,Karma Resorts,Other,Past Guests,Karma Bavaria,PF334661,Frau,Melanie,Haubner,Unknown,...,2025-11-02,1,OTA,OTA,RETAIL,2,1,0,Email and Phone,NO


In [39]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 56
Jumlah Data Yang Duplicate : 9
Jumlah Data Setelah Hapus Duplicate : 47
Jumlah Data Yang Termasuk Member: 4
Jumlah Data Setelah Hapus Member : 43
Jumlah Data Akhir : 43


In [40]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)